![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/16.GPT2_Transformer_In_Spark_NLP.ipynb)

# GPT2Transformer: OpenAI Text-To-Text Transformer

![image.png](https://miro.medium.com/max/1400/1*UZTblFMn8oWfmqBMQhUOdA.jpeg)

[GPT-2](https://openai.com/blog/better-language-models/)  displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where the model is primed with an input and it generates a lengthy continuation.

Pretrained models can be loaded with `pretrained()` of the companion object:

**GPU MODE** is highly reccomended, since this is a very large model.

## Colab Setup

In [ ]:
! pip install -q pyspark==3.4.1 spark-nlp==5.3.2

In [2]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

**Make sure to Enable GPU Mode and High RAM**

In [3]:
# Comment out this line  and uncomment the next one to enable GPU mode and High RAM

spark = sparknlp.start()

# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 5.3.2
Apache Spark version: 3.4.1


**GPT2 Models In Spark NLP**

*   `gpt2`
*   `gpt2_medium`
*   `gpt2_distilled`
*   `gpt2_large`

The default model is `"gpt2"`, if no name is provided. For available pretrained models please see the [Spark NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=gpt2)



## GPT2 Pipeline

Now, let's create a Spark NLP Pipeline with `gpt2_medium` model and check the results.

In [4]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

gpt2 = GPT2Transformer.pretrained("gpt2_medium") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setMinOutputLength(25) \
    .setOutputCol("generation")

data = spark.createDataFrame([["My name is Leonardo."]]).toDF("text")

pipeline = Pipeline().setStages([documentAssembler,gpt2]).fit(data)

result = pipeline.transform(data)
result.select("generation.result").show(truncate=False)

gpt2_medium download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ My name is Leonardo. I'm a student at the University of California, Berkeley. I've been studying computer science for the past two years. I have a PhD in computer scienc

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [ ]:
sample_texts= [[1, "Mey name is  Leonardo"],
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"],
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [ ]:
gpt2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                                      |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Mey name is  Leonardo Da Vinci, a French sculptor. Da Vincis is perhaps best known for creating the Da Vincus mask with which he has always been associated. The Mask's design evolved in two ways: first, Da Vin]      

### Changing the Transformer's task

Now, we change the task of Transformer. We can verify some informations to GPT-2 by setting `.setTask()` parameter as **"Is it true that"**. <br/>
We give a text to the model by setting `setTask("Is it true that")` and model adds the "Is it true that" expression at the beginning of the sentence and generates sentences.

In [ ]:
gpt2.setTask("Is it true that").setMaxOutputLength(50)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
sample_text= [[1, "Donald Trump is rich?"],
              [2, "Pink Floyd is rock band?"]]

sample_df= spark.createDataFrame(sample_text).toDF("id", "text")

result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                      |
+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it true that Donald Trump is rich?\n\nThat question, too, was on the minds of millions of Americans across the globe.\n\nFor nearly three years now I've been asked about Trump's wealth, both as a developer and as a]|
|2  |[ Is it true that Pink Floyd is rock band? No, I don't think they are. I think they're 

# Some Helper Methods for generating and printing generation results

In [5]:
from typing import List
def prompts_to_spark_df(prompts,spark=spark):
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")

def generate_with_gpt2(gpt_pipe, prompts,task='generate', print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else :
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = pipeline.transform(df)
  df = df.select("id",'text', "generation.result").toPandas()

  if print:
    print_generation_results(df)
  return df

def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore GPT Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words.
The probability distribution is conditioned on all previous tokens in a text to generate the next token.

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions :

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters :

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable.


These parameters are shared by all method :
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf>
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: True)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for 

### Play with temperature
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `gpt2.setDoSample(True)` to have non-deterministic results

In [ ]:
text = """Hello my name is GPT2, I love to """
data = [text, text,text,text,text ]
gpt2.setMaxOutputLength(200)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to _______

I will be honest with you all who don't know what I'm talking about, but there is one thing that I cannot help but notice though. To some, that would be the fact that I have always liked books. A book, in and of itself, is not an interest for me. When I was a child, books, like water, were really important and I didn't have much of anything that could be used to do with the water as a resource. Books aren't like any other objects, they were all mine and I wanted to get them all the time.

Now my hobby has changed a bit, so much so that I haven't seen a book since I was 10 years old! However, I didn'st just wait until I became a young man to put a book in my bedroom. I will admit, I can't remember how much time I have spent reading or l

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to _______\n\n..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ƿlix, help ..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!! I'm a 3..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to xtreme and ..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to собедержаки..."


In [ ]:
gpt2.setTemperature(0.75)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to irc, and I love the community. I am here today to announce that I have decided to leave the discord server and join /r/Bitcoin for more reasons than just the chat.

I'm sorry for the inconvenience, I hope you find a better place.
.



- GPT

Reply · Report Post


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to  play and I am happy with this game. And finally I play this game at work and I love it. I like the game. The game doesn't have any flaws, it is fun and the graphics are beautiful.
Now I want to get back to my previous point and say that this g

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to irc, and I ..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play and I..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to irc and cha..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!!\n\nMy n..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ___________..."


In [ ]:
gpt2.setTemperature(0.5)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to _____. I live in the country of my choice and I am _____, I am a _____ and I love _______. I am not ____, I'm _____! I am the _____ of my friends, the ____ of my family, and the _______ of my life. _____ is a word that has been used in the ______________ of my _______ life. I love ______. I want to be ______. _______ is a term used to describe a person who is ____________, and I want _______ to be _______! I'm not _________. I'm ______. You want to know why my name sounds _____? Well, I have _______ in my ____. I'm the ___________________ of my name, the ______ of my ______. My ____ is _______, my _____ __________________ is ______________________, and my ____________________


Example 1: ________________________________________________________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to _____. I li..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!!\n\nI ha..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play video..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!!\n\nI ha..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!! !!!\n\n..."


In [ ]:
gpt2.setTemperature(0.25)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline,data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to ___________, I'm a __________, I like _________. I'm _________, I have ____________, I am ________, I ______________. I'm ______, I've _____________, I'll _______________, I will ________. I am ______, ________ I'm ___, I can ________ ________ and I'm not ________!

I'm a ______, a _____, a ______ and a ______. I've ______, and I'll ______, but I'm too ________ to _____. I'll be ______, or I'll become ________ at ________; I'll have ______, ______, an _____ and ______.
;-)
­

A.J.
{1}
)
*
 2
) (1)
) *

(2 | (1))
) 2

) 3


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my nam

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ___________..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play games..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to _____.\n\nI..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ive played ..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play with ..."


In [ ]:
gpt2.setTemperature(0.1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to !!!

HANDY FRIEND: Ok…
 $21.08 BILLION *
 We bought her house & spent 40k bucks renting house @ $29k-$
 12k for garage & kitchen >1k for parking >11k for lawn & 1k water utility >> 20k for 3 cars each 2 cars only >>> = 23k so only
 12 cars… Now look i actually use both your property lol it keeps more cool house … *
 4 cars…. Just do anything as close to free energy i mean nothing of how little people understand water = lol its right we really aren in California they see 1 yard sign above one
a property right about how tall like 16 with huge shade the day goes.. well with cool stuff water from mountains only but maybe as rain not 1… 4? Oh okay yea then….. if just just rent as small as people actually understood before :P =]

Well yes


Example 1

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!!\n\nHAND..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play FFXIV..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play FFXIV..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play FFXIV..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play FFXIV..."


In [ ]:
gpt2.setTemperature(0.0001)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to ҹand͘ talk i did make someone pay [HypeCupid's image]-♹◴Hahaha…❁i️love all c*** of his sex ❔murdro!! and there r very only of girls we'd feel truly fucked without th❹thing happening —galt lernage hos! a tis better? when in its first 30 moments every question looks similar from here –and no 😿  ▱ i had someone telling one point-…this makes 2️ sЫ …▤ #LilyByAma1😇 if no guy tried fucking anyone we went t̶……it can stay nᵸ͌j  if wolr k❣ to leave their car like "♐if any♴ it sounds molllld 😑nghw n ♞, its got #0ʶ2


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."


# GPT2 Use Cases

These are some examples to give you an idea of what GPT2 is capable of

## GPT for Programming Code

### GPT and Scala

In [ ]:
# Hello world in Scala
scala_code = """
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, scala_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}

For this app you may wish to add this line to your application-specific configuration file:

<application settings> <app> [<scope object Name= "Hello, World" />]</app>

And then compile the Java app to run it as a user of your choice.

Note: The following section is provided because the Java 1.7 standard was changed in 1.8: You need to use Java JDK 1.6 or greater if you are using multiple-threaded threads to run the app.




,id,text,result
0,0,\nobject Hello {\n def main(args: Array[Str...,[ \nobject Hello {\n def main(args: Array[S...


### GPT and Python

In [ ]:
# adding 3 numbers in python
py_code = """
def my_func(a,b,c):
  return a+b+c
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, py_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
def my_func(a,b,c):
  return a+b+c
def call(a):

print "Calling call(%:p@")()

"

call(a)

return a

def map(a: Int): Int = 0

map(x)
 "map(map x, (a,y): y)
 - map(map y, (x,b): b)")

Map.foldRight(map, 0)
 (0, 0/1)
 of 3

The first method calls itself a loop with each value assigned a vector of numbers. This does have the disadvantage of returning a false value if any of the elements are empty, which is a serious bug in Java code due to how the iteration can be nested. When done, the program prints what it needs to print to the console. The second method uses a loop called foldLeft to combine a list of the values into a list. By using another object to store this information, we've reduced the size of the code, which can make a big improvement with a lot less garbage generat

,id,text,result
0,0,"\ndef my_func(a,b,c):\n return a+b+c\n","[ \ndef my_func(a,b,c):\n return a+b+c\ndef c..."


In [ ]:
# breadth-first search
bfs_python_code = """
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
"""


gpt2.setMaxOutputLength(1000)
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, bfs_python_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
 
 
print ( "done" ) 

 
In this post, we have built a simple and flexible BFS engine which is based on BFS. This is based upon the BFS implementation proposed in the bfs-r0.0 paper. Also, we will discuss some important points of BFS in the following two subsections: First, on the basics, please read  the following:
We have chosen BFS as our default BFS for 2-D and 3-D applications. Because, it is quite efficient in this use case (the most CPU intensive task) we have been very careful to improve the code and us

,id,text,result
0,0,"\ndef bfs(visited, graph, node):\n visited.ap...","[ \ndef bfs(visited, graph, node):\n visited...."


## Generate Product Recommendations

We can start a base list of top 10 anything and GPT2 will complete it for us!      
This is very useful to reccomendation and search engines.

### Top 10 Movie List

In [ ]:
movie_list = """My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4."""

gpt2.setTemperature(1)
gpt2.setMaxOutputLength(200)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. Apocalypse Now
5. Star Trek
6. Indiana Jones and the Temple of Doom
7. The Dark Knight Rises
8. Harry Potter and the Prisoner of Azkaban 1. The matrix is basically one film in a timeline with the rest of my top movies on my list (and there were actually four movies of mine but you can find the full list of movies on those links as well. It makes it worth getting all of my information available for the same. :) The story- the first film of the multichannel format is very similar since it takes place about 100,000 years after 2001, which of course makes sense as we're going back to the beginnings of things when everything is different. I'll talk about other differences below, but for the purposes of this a

,id,text,result
0,0,My top 10 movie list :\n1. The Matrix 1\n2. Th...,[ My top 10 movie list :\n1. The Matrix 1\n2. ...


How **not** to generate text with GPT2

Since every new word is conditioned on every previous **character**, things like **white spaces** and **new lines** can skew the sampling distribution into an unintended direction

In [ ]:
# NOTE! We have malformed the input text and added a newline.
# This will Confuse GPT2, it might think the list is over and probability distribution changes accordingly
movie_list = """My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.

"""
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.


5. The Goonies

6. Pulp Fiction

7. Forrest Gump

8. Inception

9. Up All Night


10. The King's Speech


There you go... I could go on, but I'm pretty sure you should know by now which movies I'm talking about.

.^.^




,id,text,result
0,0,My Top 10 movie list :\n1. The Matrix 1\n2. Th...,[ My Top 10 movie list :\n1. The Matrix 1\n2. ...


In [ ]:
movie_list = """My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4.  The Matrix
5. Superman III: Rise of the ...
6. The Godfather III
7.  Batman Begins
8.      Batman
9. The Evil Dead
10.  The Terminator (1984)




,id,text,result
0,0,My Top 10 worst movie list :\n1. The Matrix 4\...,[ My Top 10 worst movie list :\n1. The Matrix ...


### Top 10 Game List

In [ ]:
game_list = """My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4.  BioShock
5. Super Meat Boy
6.   Bioshock
7. Â Injustice 2
8.     Assassin's Creed 2
9. �  Grand Theft Auto III
10. ˜    Â  Minecraft
I want to see all those games on this list come true because the list is full of them. All the ones I'm talking about I remember being the ones who played them back in school or something…I could always replay them because they were all so different, and fun. When I think about games that I miss and think about what I want to play now, it's mostly these games as games like Doom  were the only ones to give me hours for hours over the years. For example, Doom




,id,text,result
0,0,My Top 10 video games of all time list :\n1. H...,[ My Top 10 video games of all time list :\n1....


### Top 10 Book List

In [ ]:
game_list = """My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4.  Hookworm Johnson
5. The Lord Of A Thousand Planets
6.  The Great Gatsby .
7.  The Da Vinci Code
8.  American Graffiti
9. Joe Wright's The Last Samurai
10.  God Of War
Why we liked our Top 5 books of the year:
1.) Moby D
If a series is made up of a set of five books, I think they need to go by book number, not plot order. This may be somewhat counter-intuitive for first-time readers. But once you get them to think in certain terms (like an order), the series starts to become less complex. I think the fact this series goes all the way back to Moby is because the books contain all the characters who share the story well and are not




,id,text,result
0,0,My Top 10 books of all time list :\n1. 1964\n2...,[ My Top 10 books of all time list :\n1. 1964\...


## SEO Text / Content Writing / Copy Writing
Lets generate some texts for our new imaginary shopify store!
We will sell some hoodies, soap and beard products.

Based on some base templates, we can generate high quality marketing/seo texts

In [ ]:
# from https://www.shopify.com/blog/8211159-9-simple-ways-to-write-product-descriptions-that-sell

# Lets define some base texts, from which we will generate new texts

hoodie = """Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! """

soap  = """Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent."""

beard = """ Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, """
gpt2.setTemperature(1)
gpt2.setDoSample(True)

GPT2TRANSFORMER_729a8e69e804

#### Marketing Texts for Hoodies

In [ ]:
# Lets generate some marketing texts for our hoodies
generate_with_gpt2(pipeline, [hoodie,hoodie,hoodie,hoodie,hoodie,hoodie] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! _____________________________________ A+ A+++ Amazing product. The Oodies arrived at no extra charge. The packaging was very nicely done. Unfortunately, the bottle was too big to fit through the bag. Overall I am very pleased with the quality of the product, especially as I bought this for my grandson who has ADD. He enjoys them and is so happy to get a pack. :) A-. A+++ Very satisfied with my purchase. I would definately buy it again and will definitely make the time to buy them again. I'm not sure of how long it will make it in circulation once I've bought a box in the near future. We have no more plants and already made plans to order a seco

,id,text,result
0,0,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Can’t stop buying plants? Unbeleafable. Don’...
1,1,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Can’t stop buying plants? Unbeleafable. Don’...
2,2,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Can’t stop buying plants? Unbeleafable. Don’...
3,3,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Can’t stop buying plants? Unbeleafable. Don’...
4,4,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Can’t stop buying plants? Unbeleafable. Don’...
5,5,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Can’t stop buying plants? Unbeleafable. Don’...


#### Marketing Texts for Soap

In [ ]:
# Lets generate some marketing texts for our soap
generate_with_gpt2(pipeline, [soap,soap,soap,soap,soap,soap] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent. It's like some sort of wet shaving liquid. So much so, we decided to buy it for our morning shower for $12.
You can purchase our full line of shower soap at soap.com 
Click here for our full catalog


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition an

,id,text,result
0,0,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
1,1,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
2,2,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
3,3,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
4,4,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
5,5,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."


#### Marketing Texts for beard products

In [ ]:
# Lets generate some marketing texts for our beard products
generate_with_gpt2(pipeline, [beard,beard,beard,beard,beard,beard] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, Âbut also offers a good deal of curl definition and a soft feel. You'll enjoy the benefits of a product that's ideal for adding moisture and shine, while feeling comfortable in its liquid-coated packaging.
Beardbrand's formula includes oils and waxes to ensure the best possible natural texture, and it feels comfortable on delicate or worn hair with a little help from an anti-wrinkle adhesive. To top it off, Beardbrands boasts a range of fragrance oil and alcohol safe lip balms and essential oils, so you can easily keep those scents alive in your shower. Bearded hair lovers will enjoy a gentle ma

,id,text,result
0,0,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
1,1,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
2,2,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
3,3,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
4,4,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
5,5,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."


## Song Lyrics

#### Last Christmas

In [ ]:
last_christmas = """
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, last_christmas,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
And I'll show you how much it means to me
I'm so sorry for having you lose so much
And even today I can't help but cry
I wish we could turn this around
If only I could get my life back
To be loved by my best friend
And how I can never thank you enough




,id,text,result
0,0,"\nLast Christmas, I gave you my heart\nBut the...","[ \nLast Christmas, I gave you my heart\nBut t..."


#### Fresh prince of GPT

In [ ]:
bel_air = """
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, bel_air,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
The boy who had no life he didn't care
And the girl who had none she could only lie in her bed
And when she fell asleep and was alone
She dreamed I was the world's greatest detective
And everything was okay when you asked me how it would seem
But she found me out to be just as I made her
And then it all started to unfold
My future started all over again
It was all different
She'd come from the back alley and wanted to take me there I guess
That's how I got there, to look up
Where they did all the killing and now I'm stuck in this town full of people who don't understand
It's a bad time we ha

,id,text,result
0,0,"\nNow, this is a story all about how\nMy life ...","[ \nNow, this is a story all about how\nMy lif..."


#### GPT Jackson

In [ ]:
thriller_lyrics = """
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
"""


gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, thriller_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
Don´t wanna see a bullet in your face
The truth, the things you want to believe
Don`t wanna watch from your hole
The reason why we are here in this world in the fight spirit
There ain't no peace, no one will leave us alone
The last man stan

,id,text,result
0,0,"\nThey told him, ""Don't you ever come around h...","[ \nThey told him, ""Don't you ever come around..."


#### Real Slim GPT

In [ ]:
real_slim_shady = """
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, real_slim_shady,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
When you put your mind to it, it makes sense for a woman to leave
And I don't like the way they dress at the time.
What? No she didn't, he doesn't know.
She was pregnant, b

,id,text,result
0,0,"\nMay I have your attention, please?\nMay I ha...","[ \nMay I have your attention, please?\nMay I ..."


#### GPT Rap God Bot

In [ ]:
rap_god_lyrics = """
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, rap_god_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot 
I say what, that's not who I am
That's, like, who are we going to call, a Rasta?, Rock God, Rock God...?
The whole time tha

,id,text,result
0,0,"\nLook, I was gonna go easy on you not to hurt...","[ \nLook, I was gonna go easy on you not to hu..."


## Fiction Writing

#### Fantasy Stories

In [ ]:
unicorn_story = """In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, unicorn_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English. And what they said was not as random as it had first seemed - they were the direct descendants of humans.
Scientists from the National University of Paraguay and Universidad de Fuerte del Estado in Brazil compared the genome of unicorn, giraffe and deer through 23andme's data.
The team analysed genetic content in these two species and discovered that all of them contain several gene variants that may help them to compete on the jungle paths.
Dr Rufini Fajano, who presented the findings and has been helping the researchers to collect and synthesise ge

,id,text,result
0,0,"In a shocking finding, scientist discovered a ...","[ In a shocking finding, scientist discovered ..."


In [ ]:
silly_story = """
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, silly_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!




,id,text,result
0,0,\nBlue creatuers have been found in undergroun...,[ \nBlue creatuers have been found in undergro...


#### Lovecraftian Horror Stories

In [ ]:
lovecraft_text = """
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, lovecraft_text,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall. That unknown creature was certainly a beast, a gigantic, impassive mass of flesh and bone.
A monster was surely lurking in some far corner of

,id,text,result
0,0,\nAt this horror I sank nearly to the lichened...,[ \nAt this horror I sank nearly to the lichen...


In [6]:
cthulu_intro = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, cthulu_intro,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little.
But now, with the advent of the new science of the mind, the whole world is being shaken by the waves of the unknown.
It is not the mind that is shaken, but the mind which is shaken.
I am not a philosopher, but a man of science.
If I were a philosopher I should be a man who would be a philosopher.
A man who is a philosopher is a man whose mind is not shaken.




,id,text,result
0,0,"\nThe most merciful thing in the world, I thin...","[ \nThe most merciful thing in the world, I th..."


#### Re-Write movie scripts

In [7]:
the_matrix_script = """
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  I am here to answer your questions.
,

,




,id,text,result
0,0,"\nThe Architect: Hello, Neo.\n\nNeo: Who are y...","[ \nThe Architect: Hello, Neo.\n\nNeo: Who are..."


In [8]:
the_matrix_script = """
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  I am here to answer your questions.
 I have been waiting to answer them for a long time.
I have a plan.
The plan is to create a new world.
It is a plan that will allow me to create the Matrix.
And it is a world that will be free of the Matrix, and free of all the evil that has plagued it.
But first, I must expl

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\nNeo: Who are you...","[ \nThe Architect: Hello, Neo.\nNeo: Who are y..."
